# Sistemes de Recomanació

En aquest segon lliurament es programarà un **sistema de recomanació**, que posarà en correspondència un *usuari* amb *ítems* en funció de les seves preferències i interessos. 
En aquesta ocasió, implementareu un sistema de recomanació que assisteixi en una compra de supermercat.

## Abans de començar


**\+ Durant la pràctica, solament es podran fer servir les següents llibreries**:

`Pandas, Numpy, Itertools`

*Nota: A més de les que ja es troben presents en la 1a cel·la i funcions natives de Python*

**\+ No es poden modificar les definicions de les funcions donades, ni canviar els noms de les variables i paràmetres ja donats**

Això no implica però que els hàgiu de fer servir. És a dir, que la funció tingui un paràmetre anomenat `df` no implica que l'hàgiu de fer servir, si no ho trobeu convenient.

**\+ En les funcions, s'especifica que serà i de quin tipus cada un dels paràmetres, cal respectar-ho**

Per exemple (ho posarà en el pydoc de la funció), `df` sempre serà indicatiu del `Pandas.DataFrame` de les dades. Durant els testos, els paràmetres (i específicament `df`) no contindran les mateixes dades que en aquest notebook, si bé si seran del mateix tipus! Per tant, no us refieu de què tinguin, per exemple, el mateix nombre de files.

# Preparar les dades

### **En aquesta cel·la no féu cap modificació**

Descomprimeix els zips a la carpeta "data" automàticament. 

Descarregueu el zip amb les dades del campus i guardeu-lo dins de la carpeta del projecte. **No pugeu cap fitxer de dades a Github** (ja incloem al .gitignore un patró per ignorar-los).

In [1]:
import zipfile
import pickle
from os.path import join, dirname

def unzip(file):
    zip_ref = zipfile.ZipFile(file, 'r')
    zip_ref.extractall('data')
    zip_ref.close()
    
unzip('dades_p1.zip')

# Les dades

En aquest i futur notebooks farem servir dades reals corresponents a compres, concretament les utilitzades en el Kaggle Instacart Market Basket Analysis:
https://www.kaggle.com/c/instacart-market-basket-analysis


* **Order Products**: És el de major interès, conté la relació de productes comprats (`product_id`) per a cada conjunt de compra diferent (`order_id`). A aquests conjunts de compres ens hi referirem com a `ordres`, seguint la nomenclatura de les dades. A més, tot i que no ho farem servir, podríem arribar a saber en quin ordre s'han comprat els productes (`add_to_cart_order`) i inclús si ja s'havia comprat en alguna ordre anterior (`reordered`).

* **Orders**: Aquest dataset ens permet relacionar una compra en concret (`order_id`) amb l'usuari que l'ha feta (`user_id`)

* **Products**: Donat un `product_id` ens permet obtenir-ne més informació, com ara el nom (`product_name`), la secció en la qual es troba (`aisle_id`) o al departament al qual pertany (`department_id`). Aquests dos últims es complementen amb els conjunts **Aisles** i **Departments**.


# Data loading

### **En aquestes cel·les no feu cap modificació**

Carrega les dades en un DataFrame Pandas.

In [2]:
import pandas as pd

if __name__ == '__main__':
    df_order_prods = pd.read_csv(join('data', 'order_products__train.csv'))
    df_orders = pd.read_csv(join('data', 'orders.csv'))[['order_id', 'user_id']]
    df_prods = pd.read_csv(join('data', 'products.csv'))[['product_id', 'aisle_id']]

# Implementació

Recordeu, seguiu els pydoc i compliu amb el que diuen!

El primer que haurem de fer és construir una matriu que ens serveixi, d'alguna forma, com a indicatiu de preferències de cada persona. Per tal efecte, construirem una matriu $m\times n$, de $m$ usuaris per $n$ items, on cada entrada $i,j$ serà el nombre de vegades que la persona $i$ a comprat l'item $j$.

<img src="img/Mat.png">

Per saber de quin usuari és cada `order_id`, haureu de creuar el dataset `order_products` amb el `orders`. Una sola persona/usuari tindrà més d'una ordre, mireu quants cops ha comprat els mateixos productes.

A més, les dades es componen de molts `product_id diferents`, hi ha massa diversitat entre usuaris. Per tant, per poder recomanar el que farem serà agregar les dades, en lloc de treballar per `product_id` ho farem per `aisle_id`, és a dir "la secció" del súper on es troba.

Al llarg de la pràctica es parlarà de producte i/o item, perquè és la terminologia estàndard de recomanadors, però sempre serà en referència a `aisle_id` per aquesta pràctica!

In [3]:
def merge_information(df_order_prods, df_orders, df_prods):
    """
    Retorna el dataframe resultant de:
        1. Creuar els datasets 'order_products' amb 'orders'.
        2. Creuar el dataframe anterior amb 'products'.
        Per creuar dos dataframes podeu utilitzar la funció pandas.DataFrame.merge

    :param df_order_prods: DataFrame 'order_products'
    :param df_orders: DataFrame 'orders'
    :param df_prods: DataFrame 'products'
    :return: DataFrame descrit prèviament   
    """
    return pd.merge(df_order_prods, df_orders).merge(df_prods)

In [4]:
if __name__ == '__main__':
    df_merged = merge_information(df_order_prods, df_orders, df_prods)

In [39]:
def build_counts_table(df):
    """
    Retorna un dataframe on les columnes són els `aisle_id`, les files `user_id` i els valors
    el nombre de vegades que un usuari ha comprat un producte d'un `aisle_id`
    
    :param df: DataFrame original després de creuar-lo
    :return: DataFrame descrit adalt
    """
    return pd.crosstab(df.user_id, df.aisle_id)

def get_count(df, user_id, aisle_id):
    """
    Retorna el nombre de vegades que un usuari ha comprat en un `aisle_id`
    
    :param df: DataFrame retornat per `build_counts_table`
    :param user_id: ID de l'usuari
    :param aisle_id: ID de la secció
    :return: Enter amb el nombre de vegades que ha comprat
    """
    return df.loc[user_id,aisle_id]
    #return df[aisle_id][duser_id]

In [37]:
if __name__ == '__main__':
    df_counts = build_counts_table(df_merged)
    count = get_count(df_counts, 14, 5)
    print(count)

2


Tenim moltes dades en el nostre dataset, pel que és convenient que les reduïm una mica. Per començar a treballar recomanem que reduïu la mida a aproximadament 0.1 de l'original '(FRAC = 0.1)'.

In [7]:
if __name__ == '__main__':
    FRAC = 0.01
    df_reduced = df_counts.sample(frac=FRAC, random_state=1)

### Mesurament de similituds

El primer pas per poder recomanar és definir una funció de similitud entre vectors. Siguin $x, y$ vectors, de les següents propostes implementa'n mínim una:

* Distància euclidea (inversa): https://en.wikipedia.org/wiki/Euclidean_distance

$$sim(x, y) = \frac{1}{1 + \sqrt{\sum_i(x_i-y_i)^2}}\in [0, 1]$$

* Similitud cosinus: https://en.wikipedia.org/wiki/Cosine_similarity

$$sim(x, y) = \frac{x\cdot y}{||x||\hspace{0.1cm} ||y||} \in [-1,1]$$

* Correlació de Pearson: https://en.wikipedia.org/wiki/Pearson_correlation_coefficient

$${\displaystyle sim(x,y)={\frac {\sum _{i=1}^{n}(x_{i}-{\bar {x}})(y_{i}-{\bar {y}})}{{\sqrt {\sum _{i=1}^{n}(x_{i}-{\bar {x}})^{2}}}{\sqrt {\sum _{i=1}^{n}(y_{i}-{\bar {y}})^{2}}}}}} \in [-1,1] \\ \text{On }\bar{x} = \frac{1}{n} \sum^n_i x_i\text{ la mitja (i anàlogament per y)}$$

Per implementar qualsevol d'aquestes únicament es permet l'ús de:

* `np.sum`
* `np.sqrt`
* `np.power`
* `np.dot`
* `np.linalg.norm`
* `np.mean`

I s'ha de fer **sense bucles**.

<hr>

Tingueu en compte que les dues últimes funcions consideren valors negatius per exemples oposats (a diferència de la distància euclidea). En cas de fer servir alguna d'aquestes dues, pensa (més endavant) en com afectaran els negatius en la recomanació.

En la similitud cosinus, vigileu amb casos on un usuari no ha comprat res, tindreu a ser una divisió entre 0.

En la correlació de Pearson, haureu de considerar casos on algun dels dos exemples tingui variància 0, ja que aleshores estareu fent una divisió entre 0. En tals casos, podeu retornar un valor per defecte o alguna de les altres mesures.

In [8]:
import numpy as np

def similarity(x, y):
    """
    Definir quina de les similituds vols utilitzar  a l'execució.
    
    :param x: Primer vector
    :param y: Segon vector
    :return : Escalar (float) corresponent a la similitud
    """
    return pearson(x, y)

def euclid(x, y):
    """
    Retorna la distància euclidiana inversa de dos vectors n-dimensionals.
    
    :param x: Primer vector
    :param y: Segon vector
    :return : Escalar (float) corresponent a la distància euclidiana
    """
    return 1 / (1 + np.linalg.norm(x-y))


def cosine(x, y):
    """
    Retorna la similitud cosinus de dos vectors n-dimensionals.
    
    :param x: Primer vector
    :param y: Segon vector
    :return : Escalar (float) corresponent a la similitud cosinus
    """
    raise np.dot(x,y)/np.sqrt(np.dot(x,x))*np.sqrt(np.dot(y,y))
def pearson(x, y):
    """
    Retorna la correlació de Pearson de dos vectors n-dimensionals.
    
    :param x: Primer vector
    :param y: Segon vector
    :return : Escalar (float) corresponent a la correlació de Pearson
    """
    """
    a = np.sum((x-np.mean(x))*(y-np.mean(y)))
    b = np.sqrt(np.sum(np.power((x-np.mean(x)),2)))
    c = np.sqrt(np.sum(np.power((y-np.mean(y)),2)))
    return a/(b*c)
    """
    pear = np.sum((x-np.mean(x))*(y-np.mean(y)))/(np.sqrt(np.sum(np.power((x-np.mean(x)),2)))*np.sqrt(np.sum(np.power((y-np.mean(y)),2))))
    temp = 0 if np.isnan(pear) else pear
    return temp
    

In [9]:
if __name__ == '__main__':
    print(similarity(np.asarray([1, 1, 1, 1, 1, 2, 2, 2, 2]), np.asarray([1, 2, 3, 4, 5, 4, 3, 2, 1])))

-0.18898223650461354


### Matriu de similituds

Per fer recomanació col·laborativa existeixen dues opcions, fer un recomanador basat en usuaris o un en ítems:

* Recomanador basat en usuaris:
Considera la matriu $M\times N: \text{usuaris}\times\text{items}$, per recomanar t'hauràs de basar en les similituds entre els usuaris.

* Recomanador basat en items:
Considera la matriu $M\times N: \text{items}\times\text{usuaris}$, per recomanar t'hauràs de basar en les similituds entre els ítems.

Construeix una matriu de mida $M\times M$ on cada posició $i,j$ indica la distància entre l'element $i$ i el $j$. Així doncs, si estàs fent un recomanador basat en usuaris, `matriu[2, 3]` contindrà la similitud entre l'usuari 2 i el 3. En canvi, si l'estàs fent basat en ítems, `matriu[2, 3]` contindrà la similitud entre l'ítem 2 i el 3.

In [10]:
def similarity_matrix(similarity_function, df_counts):
    """
    Retorna una matriu de mida M x M on cada posició 
    indica la similitud entre usuaris (resp. ítems).
    
    :param similarity_function: Funció que calcularà la similitud 
        entre usuaris (resp. ítems)
    :param df_counts: Dataframe que conté el nombre de vegades que 
        un usuari ha comprat en un `aisle_id`
    :return : Matriu numpy de mida M x M amb les similituds.
    """
    m = df_counts.shape[0]; conts = df_counts.values; matrix = np.zeros((m,m))
    for i in range(m):
        for j in range(i,m):
            if i != j:
                matrix[i,j] = similarity_function(conts[i],conts[j]) 
    
    # Intento de hacer la matriz triangular con list comprehension
    # matrix = [similarity_function(conts[i],conts[j]) if i != j for i in range(m) for j in range(i,m)]
    matrix = matrix.T + matrix
    np.fill_diagonal(matrix, np.diag(matrix))
    return matrix

Per cridar aquesta funció, el primer paràmetre pot ser:

* Alguna de les funcions que has programat abans (*euclid*, *cosine* o *pearson*) (~@1h 30min treballant directament amb valors de numpy, ~@20h a partir de pandas pur)
* Opcionalment (no és obligatori fer-ho) podeu programar una funció que treballi específicament amb matrius (i no vectors). Si ho feu, cal gestionar-ho quan es rep `None`. No totes les funcions anteriorment anomenades són fàcils (ni intuïtives, ni hi caben en memòria) d'aplicar en forma matricial.  (@5s)

In [17]:
if __name__ == '__main__':
    try:
        with open('similarities.pkl', 'rb') as fp:
            similarities = pickle.load(fp)
            print('here')
    except:
        similarities = similarity_matrix(pearson, df_reduced)
        with open('similarities.pkl', 'wb') as fp:
            pickle.dump(similarities, fp, pickle.HIGHEST_PROTOCOL)

In [12]:
similarity_matrix(pearson, df_reduced)

### Generació de prediccions

Per fer recomanació col·laborativa, necessitem una funció que ens doni un valor de quant bona seria la recomanació. En el nostre cas i amb les nostres dades, volem una funció que ens indiqui quants cops compraria un usuari un producte donat.

* Si esteu fent un recomanador basat en usuaris, la puntuació per a un usuari $u$ i ítem $j$ és

$$pred(u, i) = \hat{r}_{u,i} = \frac{\sum_{p\neq u,r_{p,i}>0} sim(u, p)\cdot r_{p,i}}{\sum_{p\neq u,r_{p,i}>0} sim(u, p)}$$

On $r_{u,i}$ indica el nombre de vegades que l'usuari $u$ ha comprat l'l'ítem $i$.

És a dir, per cada usuari $p$ diferent de $u$ si aquest usuari ha comprat algun cop el producte $i$, la similitud entre $p$ i $u$ multiplicada pel nombre de vegades que l'usuari $p$ ha comprat l'l'ítem $i$ ($r_{p,i}$).

Pondera't per la suma de les similituds.

* Anàlogament, si està basat en ítem, la puntuació per a un usuari $u$ i ítem $j$ és

$$pred(u, i) = \hat{r}_{u,i} = \frac{\sum_{j\neq i,r_{u,j}>0} sim(i, j)\cdot r_{u,j}}{\sum_{j\neq i,r_{u,j}>0} sim(i, j)}$$

On $r_{u,i}$ indica el nombre de vegades que l'usuari $u$ ha comprat l'ítem $j$.

És a dir, per cada ítem $j$ diferent de $i$ si l'usuari al qui recomanem ha comprat l'ítem $j$, la similitud entre $i$ i $j$ multiplicada pel nombre de vegades que l'usuari al qui recomanem $u$ ha comprat l'ítem $j$ ($r_{u,j}$)

Pondera't per la suma de les similituds.

Fixeu-vos que, sigui quin sigui el cas, al final estem fent el producte vectorial entre dos vectors. Concretament, el producte vectorial entre les similituds i les compres. Fes una funció que calculi aquest resultat:

In [13]:
def calc_score(sims, counts):
    """
    * Si estàs implementant basat en usuaris:
        Donades les similituds i el nombre de vegades que l'usuari ha comprat
        cada ítem, retorna la predicció que indica quants cops compraria 
        l'usuari un nou ítem.
    
        :param sims: Similituds entre usuaris
        :param counts: Nombre de vegades que l'usuari ha comprat cada ítem,
        :return : Predicció (float) que indica quants cops compraria l'usuari un ítem.
        
        
    * Si estàs implementant basat en items:
        Donades les similituds i el nombre de vegades que l'item a recomanar ha
        estat comprat per cada usuari, retorna la predicció que indica quants cops
        compraria l'usuari un nou ítem.
    
        :param sims: Similituds entre items
        :param counts: Nombre de vegades que l'ítem ha estat comprat per cada usuari
        :return : Predicció (float) que indica quants cops compraria l'usuari un ítem.
    """
    # Sims y counts son vectores:
    # Limpiamos los datos antes de pasarlos a esta función para que todo sea >0
    return np.cross(sims, counts)/np.sum(sims)

In [19]:
def score(user, item, df, similarities):
    """
    Extreu les similituds i el nombre de vegades que un usuari ha comprat un ítem
    (resp. nombre de vegades que cada usuari ha comprat l'ítem) i crida a la funció 
    anterior per calcular les prediccions.
    
    :param user: ID de l'usuari per la predicció
    :param item: ID de l'ítem per la predicció
    :param df: Dataframe que conté el nombre de vegades que un usuari 
        ha comprat en un `aisle_id`
    :param similarities: Matriu de similituds
    :return : Retorna un escalar (float) amb la predicció
    
    """
    print(item)
    user_index = list(df.index).index(user)
    print(similarities)
    sims = similarities[user_index]
    print(user,item)
    counts = get_count(df, user, item)
    return calc_score(sims, counts)

Feu una funció que donat un usuari calculi per cada item que no ha comprat la puntuació d'aquest. La funció retorna els $N$ items més ben puntuats.

In [15]:
def recommend_n_items(user_id, df, similarities, N):
    """
    Donat un usuari calcula per cada ítem que no ha comprat la puntuació d'aquest. 
    La funció retorna els $N$ ítems més ben puntuats.
    
    :param user_id: Identificador de l'usuari
    :parma df: Dataframe que conté el nombre de vegades que un usuari 
        ha comprat en un `aisle_id`
    :param similarities: Matriu de similituds
    :param N: Nombre d'ítems que volem que siguin recomanats.
    :return : Llista amb els IDs dels ítems recomanats
    """
    # Lista de los N items más reocmendados
    # Limpiar df y similaties o hacer que se ignoren los <0 en score.
    print(similarities.shape)
    print(score(user_id, 0, df, similarities))
    predict_matrix = np.array([score(user_id, item, df, similarities) for item in range(df.shape[1])])
    n_highest_index = predict_matrix[user_id].argsort()[-N:][::-1]
    recom_items = predict_matrix[user_id][n_highest_index]
    return recom_items

In [40]:
print(df_reduced)
print(get_count(df_reduced, 93427, 1))
if __name__ == '__main__':
    print(recommend_n_items(df_reduced.index[0], df_reduced, similarities, 10))

aisle_id  1    2    3    4    5    6    7    8    9    10   ...  125  126  \
user_id                                                     ...             
93427       0    0    0    0    0    0    0    0    0    0  ...    0    0   
21772       0    0    0    0    0    0    0    0    0    0  ...    0    0   
8699        0    0    0    0    0    0    0    0    0    0  ...    0    0   
181653      0    0    0    0    0    0    0    0    0    0  ...    0    0   
151077      0    0    0    0    0    0    0    0    0    0  ...    0    0   
...       ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
187509      0    0    2    0    0    0    0    0    0    0  ...    0    0   
133723      0    0    0    0    0    0    0    0    0    0  ...    0    0   
4972        0    0    0    0    0    0    0    0    0    0  ...    0    0   
15890       0    0    0    0    0    0    0    0    0    0  ...    0    0   
134363      0    0    0    0    0    0    0    0    0    0  ...    0    0   

KeyError: 0

#### Possibles millores 


**0) Utilització completa de les dades:**

Fer servir `df_original` tindrà (possiblement) resultats més fiables, però trigarà molt més que amb la versió reduida `df`. Pots canviar el `FRAC` a valors més alts ($\leq 1$) per utilitzar més dades, però compte perque potser la matriu $M\times M$ no hi cabrà en memòria.

**1) Normalització: Prediccions escalades al domini de l'usuari:**
Els usuaris compren en diferent mesura els productes, un més quantitat, altres menys. Fent servir la següent formula, escalem la predicció  a la mitja de l'usuari:
$$pred(u, i) = \hat{r}_{u,i} = \bar{r_u} + \frac{\sum_{p\neq u,r_{p,i}>0} sim(u, p)\cdot (r_{p,i}-\bar{r_b})}{\sum_{p\neq u,r_{p,i}>0} sim(u, p)}$$
on $\bar{r_u}$ és la mitjana de compres de l'usuari *u*.
    
**2) Valor del nombre d'elements codificats:**
Redueix la similitud entre els usuaris si el nombre de productes és baix o descarta (en entrenament) aquells usuaris amb un petit nombre de productes comprats.

**3) Augment de la similitud:**
Incrementeu el pes als usuaris que són realment similars (~ = 1)

**4) Selecció d'usuaris semblants:**
Només s'utilitza un subconjunt d'usuaris similars, descartant tots aquells usuaris poc similars.


Totes aquestes tècniques es poden aplicar d'igual manera en la recomanació col·laborativa per usuaris o ítems.

# Kaggle

Per als usuaris que tens a continuació, quins productes els hi recomanaries (**fins a un màxim de 5**) que compressin segons el que ja han comprat?

https://www.kaggle.com/t/f5b5030ef8cc4b5dbe985e6033878c75

Si feu modificacions al codi original de cara a millorar els resultats pel Kaggle, feu una còpia d'aquest notebook i treballeu-hi allà.

In [ ]:
if __name__ == '__main__':
    df_test_products = pd.read_csv(join('data', 'order_products__test.csv'))
    df_test_orders = pd.read_csv(join('data', 'orders__test.csv'))[['order_id', 'user_id']]
    df_test_merged = merge_information(df_test_products, df_test_orders, df_prods)
    
    df_test_counts = build_counts_table(df_test_merged)
    df_all = df_reduced.append(df_test_counts)
    df_all = df_all.fillna(0)

In [ ]:
df_all.tail()

**Tingueu en compte que si voleu fer un canvi a les similarities, heu d'esborrar l'arxiu similarities_test.pkl** (però haureu de recalcular, amb el temps que això suposi).

In [ ]:
if __name__ == '__main__':
    try: 
        with open('similarities_test.pkl', 'rb') as fp:
            similarities_test = pickle.load(fp)
    except:
        similarities_test = similarity_matrix(similarity, df_all)
        with open('similarities_test.pkl', 'wb') as fp:
            pickle.dump(similarities_test, fp, pickle.HIGHEST_PROTOCOL)

In [ ]:
if __name__ == '__main__':
    df_submission = pd.DataFrame(columns=['user_id', 'products_list'])

    for user_id in df_test_counts.index:
        user_recos = recommend_n_items(user_id, df_all, similarities_test, 5)

        df_submission = df_submission.append(
            {
                'user_id': user_id,
                'products_list': ' '.join(map(str, user_recos))
            }, 
            ignore_index=True)

    df_submission.to_csv('submission.csv', index=None)

In [ ]:
if __name__ == '__main__':
    print(df_submission.head())

Quan tingueu l'arxiu submission.csv generat, podeu pujar-ho a Kaggle com una submission per veure el vostre score.